<a href="https://colab.research.google.com/github/Mohinee29/Quora-Question-Pair-Similarity/blob/main/Quora_W2V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# avoid decoding problems
df = pd.read_csv("/content/drive/MyDrive/Quora/train_sample.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,398782,496695,532029,What is the best marketing automation tool for...,What is the best marketing automation tool for...,1
1,115086,187729,187730,I am poor but I want to invest. What should I do?,I am quite poor and I want to be very rich. Wh...,0
2,327711,454161,454162,I am from India and live abroad. I met a guy f...,T.I.E.T to Thapar University to Thapar Univers...,0
3,367788,498109,491396,Why do so many people in the U.S. hate the sou...,My boyfriend doesnt feel guilty when he hurts ...,0
4,151235,237843,50930,Consequences of Bhopal gas tragedy?,What was the reason behind the Bhopal gas trag...,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [ ]:
!python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_md
!python -m spacy download en

     |████████████████████████████████| 827.9 MB 1.3 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=27589850664e576d79b54528e5770fd5133aca60e7e67a8235ad85a43996246d
  Stored in directory: /tmp/pip-ephem-wheel-cache-zpexybja/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 12.0 MB 5.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import spacy.cli

spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
import spacy 
nlp = spacy.load('en_core_web_lg')
vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progrss bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)
x=nlp('man')
len(x.vector)

100%|██████████| 30000/30000 [05:15<00:00, 94.98it/s]


300

In [ ]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████| 30000/30000 [05:23<00:00, 92.80it/s]


In [ ]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('/content/drive/MyDrive/Quora/nlp_features_train.csv'):
    dfnlp = pd.read_csv("/content/drive/MyDrive/Quora/nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('/content/drive/MyDrive/Quora/df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("/content/drive/MyDrive/Quora/df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [ ]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [ ]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,398782,1,0.874989,0.874989,0.999980,0.999980,0.923070,0.923070,1.0,1.0,0.0,13.0,99,99,99,99,0.844156
1,115086,0,0.666644,0.499988,0.714276,0.624992,0.583328,0.466664,1.0,1.0,3.0,13.5,74,65,69,67,0.220000
2,327711,0,0.000000,0.000000,0.428565,0.272725,0.149999,0.115384,0.0,0.0,6.0,23.0,43,34,26,27,0.047170
3,367788,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,17.0,21.5,30,23,29,44,0.050000
4,151235,0,0.749981,0.599988,0.000000,0.000000,0.599988,0.333330,1.0,0.0,4.0,7.0,69,48,55,71,0.555556


In [ ]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,398782,1,1,76,77,12,12,11.0,24.0,0.458333,2,0
1,115086,1,1,49,57,12,15,7.0,23.0,0.304348,2,0
2,327711,1,1,105,120,25,17,2.0,34.0,0.058824,2,0
3,367788,1,1,59,146,12,30,0.0,32.0,0.000000,2,0
4,151235,1,1,35,50,5,9,3.0,13.0,0.230769,2,0


In [ ]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-12.894651,24.185946,2.913282,-11.786999,3.314275,-0.712037,3.813318,-5.872210,14.315707,112.842290,...,-30.182353,3.501638,17.139779,12.068506,-5.537776,-6.454221,-1.086545,0.934220,-3.572049,5.394783
1,-9.864155,9.241816,-8.938806,-1.926636,-5.181161,-0.544510,3.767745,-8.014432,-5.238980,108.207363,...,-15.102815,6.410637,-6.481338,-6.075532,3.525848,1.491746,-7.409518,-8.288587,3.839561,5.981986
2,-8.034306,14.148296,-11.523457,-1.998582,21.497613,-0.553386,-0.900359,-21.151714,5.586921,246.322043,...,-3.048932,3.899859,-0.181175,-7.548079,17.850292,-10.766386,-8.342056,0.954076,5.829421,12.093768
3,-6.478069,3.158590,-9.119894,-13.905647,10.521860,4.084668,-4.870337,4.323753,1.515728,143.961344,...,-12.797720,2.774519,-9.153279,-12.332205,-3.398042,4.269216,-0.785181,1.479683,2.171330,8.775641
4,-3.132839,10.427197,7.655022,5.991682,-6.872755,-1.138129,-1.562139,8.336739,-6.936201,74.340373,...,-11.115250,4.876313,10.815108,-8.928386,-1.175635,15.915021,-9.132443,6.243993,-7.642362,12.053671


In [ ]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-10.730862,17.942179,1.966947,-14.341941,6.410039,-2.712048,2.062936,-6.670762,15.883176,112.766174,...,-33.521917,4.900120,17.347702,11.634150,-5.349538,-4.069301,3.427811,-2.312354,-2.384895,7.259330
1,-15.720500,12.470273,-15.933820,0.398255,-3.525124,6.948885,11.971279,-10.894463,-9.916050,145.693046,...,-21.094213,4.433884,-9.766198,-5.465565,9.470171,10.817784,1.042753,-12.991802,2.166817,4.231539
2,-16.531340,5.279469,9.993403,-4.650677,-2.082348,-2.310146,-4.596347,-31.103522,-12.138646,159.503285,...,8.499174,-15.966543,3.993850,-7.849429,-5.492049,-7.836266,-0.272645,46.178656,13.207780,21.047092
3,-25.818861,44.188080,-39.810337,-29.340931,9.894199,-7.746117,4.200149,-57.055254,-13.013433,521.018052,...,-1.578354,-6.124876,-5.524728,-2.407836,36.537639,5.944407,-1.175200,38.982331,20.501046,2.190571
4,2.861994,11.435291,6.675223,6.184404,3.602123,0.419329,-2.390176,2.112800,-4.439770,106.263240,...,-12.053610,0.963612,16.356464,-11.359396,0.024958,7.979280,-9.142534,6.566577,-3.385568,18.179580


In [ ]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 300
Number of features in question2 w2v  dataframe : 300
Number of features in final dataframe  : 629


In [ ]:
# storing the final features to csv file
if not os.path.isfile('/content/drive/MyDrive/Quora/final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('/content/drive/MyDrive/Quora/final_features.csv')